In [1]:
import pandas as pd
import torch, torch_geometric
import torch
from torch_geometric.data import Data
from sklearn.neighbors import NearestNeighbors
from collections import defaultdict

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch_geometric.utils import to_networkx
import networkx as nx
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.nn import MSELoss
import copy
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import joblib
import re
from sklearn.neighbors import BallTree
from sklearn.metrics import pairwise_distances
from torch_geometric.nn import NNConv
from torch_geometric.typing import Adj
from torch_geometric.nn import RGCNConv
from torch.optim import AdamW
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import mean_squared_error, r2_score
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from hangul_romanize import Transliter
from hangul_romanize.rule import academic
import matplotlib
import optuna

/home/khs/anaconda3/envs/hoso/lib/python3.10/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/khs/anaconda3/envs/hoso/lib/python3.10/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  import torch_geometric.typing
/home/khs/anaconda3/envs/hoso/lib/python3.10/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /home/khs/anaconda3/envs/hoso/lib/python3.10/site-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  import torch_geometric.typing
/home/khs/anaconda3/envs/hoso/lib/python3.10/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /home/khs/anaconda3/envs/hoso/lib/python3.10/site-packages/torch_spline_con

In [2]:
# data
df = pd.read_csv("/home/khs/data/csv/matching data/chla_sentinel2_trans.csv")
print(len(df))
df

881


,분류번호,측정소명,년/월/일,회차,경도,위도,채수시각,수심(m),수온(℃),클로로필 a(㎎/㎥),...,B4,B5,B6,B7,B8,B8A,B9,SCL,idx,image_id_S2
0,5003B20,나주댐1,2018/02/22,1회차 상층부,"126°51'36""","34°57'22.39""",11:07,NaN,2.9,9.3,...,245.0,239.0,225.0,243.0,249.0,220.0,195.0,6.0,963,20180222T021709_20180222T021703_T52SBD
1,5003B10,나주댐2,2018/02/22,1회차 상층부,"126°50'55.32""","34°55'39.36""",11:23,NaN,3.7,8.3,...,511.0,629.0,590.0,628.0,712.0,556.0,473.0,6.0,964,20180222T021709_20180222T021703_T52SBD
2,5003B20,나주댐1,2018/02/22,1회차 중층부,"126°51'36""","34°57'22.39""",11:15,NaN,3.6,8.6,...,245.0,239.0,225.0,243.0,249.0,220.0,195.0,6.0,967,20180222T021709_20180222T021703_T52SBD
3,5003B20,나주댐1,2018/02/22,1회차 하층부,"126°51'36""","34°57'22.39""",11:11,NaN,3.6,8.2,...,245.0,239.0,225.0,243.0,249.0,220.0,195.0,6.0,969,20180222T021709_20180222T021703_T52SBD
4,2018B30,남강댐1(진양호),2018/10/15,1회차 상층부,"128°1'53.9""","35°10'4.8""",09:30,40.98,17.7,4.5,...,832.0,645.0,172.0,154.0,125.0,79.0,0.0,6.0,5034,20181015T021641_20181015T022428_T52SDD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,4007B70,주암댐1,2025/10/28,3회차 상층부,"127°14'26.74""","35°3'23.78""",10:10,NaN,21.2,6.0,...,76.0,66.0,51.0,52.0,49.0,62.0,69.0,6.0,38663,20251028T021821_20251028T022504_T52SCD
877,4007B70,주암댐1,2025/10/28,3회차 중층부,"127°14'26.74""","35°3'23.78""",10:10,NaN,20.0,5.8,...,76.0,66.0,51.0,52.0,49.0,62.0,69.0,6.0,38664,20251028T021821_20251028T022504_T52SCD
878,4007B70,주암댐1,2025/10/28,3회차 하층부,"127°14'26.74""","35°3'23.78""",10:10,NaN,12.7,0.4,...,76.0,66.0,51.0,52.0,49.0,62.0,69.0,6.0,38665,20251028T021821_20251028T022504_T52SCD
879,5001B40,광주댐1,2025/11/07,1회차,"126°59'9.6""","35°11'59.14""",13:57,NaN,18.5,5.3,...,217.0,157.0,75.0,58.0,78.0,61.0,35.0,6.0,38961,20251107T021911_20251107T022008_T52SCD
